In [1]:
## Retrieval augmented generation
# .env with a variable file should exist for True
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

openai_api_key = os.getenv('OPENAI_API_KEY')

# Example: Use this key to authenticate with OpenAI
import openai
openai.api_key = openai_api_key

In [3]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader
documents=SimpleDirectoryReader("./data").load_data()

In [4]:
documents

[Document(id_='65888a00-77fa-406f-9707-01a9e4ffc7ba', embedding=None, metadata={'page_label': '1', 'file_name': 'cat1.pdf', 'file_path': '/home/zen/Documents/DS/OpenAI/llamaindex-RAG/data/cat1.pdf', 'file_type': 'application/pdf', 'file_size': 402183, 'creation_date': '2024-12-23', 'last_modified_date': '2024-12-23'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text=' \n \nRESPONSIBLE CAT BREEDING GUIDELINES \n \n \n1 \nEndorsed by the EU Platform on Animal Welfare 3 November 2020 - DOC/2020/11982 Rev1. \n \n \n \n \nPlease note: This document will be a base for developing shorter and user-friendly documents \ntargeted at relevant audiences.

In [5]:
index=VectorStoreIndex.from_documents(documents,show_progress=True)

/home/zen/Documents/DS/OpenAI/llamaindex-RAG/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 56/56 [00:03<00:00, 16.68it/s]


In [6]:
index

In [7]:
query_engine=index.as_query_engine()

In [11]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.postprocessor import SimilarityPostprocessor

retriever=VectorIndexRetriever(index=index,similarity_top_k=4)
postprocessor=SimilarityPostprocessor(similarity_cutoff=0.80)

query_engine=RetrieverQueryEngine(retriever=retriever,node_postprocessors=[postprocessor])

In [12]:
response=query_engine.query("list some breeds of cat?")

In [14]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)
print(response)


Final Response: Maine Coon, American Shorthair, Persian, Sokoke,
British Shorthair, Exotic Shorthair, Norwegian Forest Cat, Siberian,
Burmese, Havana Brown, Singapura, Turkish Angora, Turkish Van,
Japanese Bobtail, Egyptian Mau.
______________________________________________________________________
Source Node 1/4
Node ID: 0c449385-b5ca-4591-af2f-5d4b3e906083
Similarity: 0.8644691510475131
Text: development of several other breeds. Crossbreeding with Persian
cats is often used to produce a more brachycephalic head type. The
Exotic Shorthair is essentially a shorthaired variant of the Persian,
which is demonstrated by the complete clustering of these two breeds
as one group. The most dolichoce- phalic breed, the Siamese, has also
been us...
______________________________________________________________________
Source Node 2/4
Node ID: 6ddbb270-b7da-4b33-b5d3-e46a0f36c973
Similarity: 0.8552687555665012
Text: The two breeds that originated in the United States (Maine Coon
and American Sho

In [18]:
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("list some eesntials topics we learn?")
print(response)


Some essential topics we learn from the provided context are responsible cat breeding guidelines, principles of responsible breeding, selection of parents based on various considerations, competent human carers, requirements for good animal welfare including feeding, housing, health, and appropriate behavior, and the importance of meeting cats' environmental needs for their well-being.
